In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as opt
import pandas as pd
from sklearn.metrics import classification_report
import scipy.io as sio

In [2]:
## 应用1VSall

In [3]:
import matplotlib

In [4]:
path = 'ex3data1.mat'
data = sio.loadmat(path)

In [5]:
def plot_image(image):
    fig,ax=plt.subplots(1,figsize=(1,1))
    ax.matshow(image.reshape(20,20),cmap = matplotlib.cm.binary)
    

In [6]:
data['X']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
X=data['X']
y=data['y']

In [8]:
X.shape

(5000, 400)

In [9]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [10]:
def cost(theta,X,y):
    return np.mean(-y*np.log(sigmoid(X@theta))-(1-y)*np.log(1-sigmoid(X@theta)))

In [29]:
def gradient(theta,X,y):
    return np.array((sigmoid(X@theta)-y)@X).ravel()/len(X)

In [43]:
from scipy.optimize import minimize

def one_vs_all(X,y,num_labels,a):
    rows = X.shape[0]
    params = X.shape[1]
    ans = np.zeros((params+1,num_labels))
    X = np.insert(X,0,values = np.ones(rows),axis=1)
    
    for i in range(num_labels+1):
        theta = np.zeros(params+1)
        theta = np.reshape(theta,(1,params+1))
        y_i = np.array([1 if label==i else 0 for label in y])
        fmin = minimize(fun=cost,x0=theta,args=(X,y_i),method='TNC',jac=gradient)
        all_theta[:,i-1] = fmin.x
    return all_theta

In [44]:
all_theta = np.zeros((X.shape[1]+1,10))


In [45]:
all_theta = one_vs_all(X,y,10,1)


<ipython-input-10-f22e899c3d08>:2: RuntimeWarning: divide by zero encountered in log
  return np.mean(-y*np.log(sigmoid(X@theta))-(1-y)*np.log(1-sigmoid(X@theta)))
<ipython-input-10-f22e899c3d08>:2: RuntimeWarning: invalid value encountered in multiply
  return np.mean(-y*np.log(sigmoid(X@theta))-(1-y)*np.log(1-sigmoid(X@theta)))
<ipython-input-9-7a57b3e79abb>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


In [46]:
all_theta

array([[-6.05726801e+00, -5.53968059e+00, -8.58433297e+00, ...,
        -1.32887557e+01, -8.68350387e+00, -1.25484541e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.84445122e-02,  4.95581166e-02, -2.55725115e-04, ...,
        -6.69606820e+00, -1.91978559e-01,  1.16414470e-03],
       [ 2.81693595e-07, -5.55392151e-03, -1.26557197e-06, ...,
         7.73348511e-01,  9.67706194e-03,  5.21182784e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [47]:
def predict(X,all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels=10
    X=np.insert(X,0,1,axis=1)
    X = np.matrix(X)
    all_theta=np.matrix(all_theta)
    h=sigmoid(X@all_theta)
    h_argmax = np.argmax(h,axis=1)
    return h_argmax+1

In [48]:
y_pred = predict(X,all_theta)

In [50]:
classification_report(y,y_pred)

'              precision    recall  f1-score   support\n\n           1       1.00      1.00      1.00       500\n           2       0.97      0.97      0.97       500\n           3       0.97      0.95      0.96       500\n           4       0.97      0.98      0.97       500\n           5       0.98      0.96      0.97       500\n           6       0.98      0.99      0.99       500\n           7       0.96      0.97      0.97       500\n           8       0.96      0.97      0.96       500\n           9       0.95      0.95      0.95       500\n          10       0.99      1.00      1.00       500\n\n    accuracy                           0.97      5000\n   macro avg       0.97      0.97      0.97      5000\nweighted avg       0.97      0.97      0.97      5000\n'